# Công cụ **tìm kiếm tương tác k-NN** cho **Span Adaptation Vectors** (Test → Train) với **trọng số lớp**

- **Mục đích**
  - Cho phép bạn **nhập đường dẫn 1 file Span AV (.pt) của mẫu Test** và nhận về **K láng giềng gần nhất** trong **DB Train** (GramVar + ParaVE) theo **điểm tương đồng có trọng số theo lớp**.
  - In ra: điểm số, nguồn (dataset/verb/arg), **câu gốc** của neighbor (lấy từ JSON Train), và tên file vector gốc.


- **Đầu vào bắt buộc**
  - Trọng số lớp đã học (softmax 12 lớp):  
    `.../matching_results/learned_layer_weights.pt`
  - Database Train đã cache:
    - Vectors (tensor): `.../cached_databases/combined_train_db_vectors.pt`  
      > Dạng `[N_spans, 12, 2304]` (12 lớp; 2304 = concat `[begin || end || content]`).
    - Metadata (json): `.../cached_databases/combined_train_db_metadata.json`  
      ```json
      {"verb_name":"interact","arg_label":"ARG-0",
       "original_file":"sentence_123_ARG_0.pt",
       "source_dataset":"span_adaptation_vectors_train_parave"}
      ```
  - Map thư mục Train gốc để **tra cứu text**:
    ```
    span_adaptation_vectors_train_gramvar -> .../Split_GramVar/Train
    span_adaptation_vectors_train_parave -> .../Split_ParaVE/Train
    ```


- **Đầu vào khi chạy**
  - Trong vòng lặp, nhập **đường dẫn file test** dạng:  
    `.../span_adaptation_vectors_test_<parave|gramvar>/<verb>/sentence_<i>_ARG_<k>.pt`

- **Đầu ra**
  - In ra console:
    - **Top-K neighbors** (K = 5 mặc định) gồm: `score`, `source_dataset / verb / arg`, **câu gốc** của neighbor (đọc từ `<verb>_train_set.json` dựa vào `sentence_<idx>`), và `original_file`.

- **Cách tính điểm (trọng số lớp)**
  - Với test vector $T \in \mathbb{R}^{12\times 2304}$ và mỗi DB vector $D_j \in \mathbb{R}^{12\times 2304}$:
    1. Tính cosine cho **mỗi lớp** $l=1..12$:

  $$\text{sim}_{j,l} = \cos(T_l, D_{j,l})$$
  
    2. Lấy trọng số lớp $w_l$ đã học (softmax, $\sum_l w_l = 1$).
    3. **Điểm cuối**:
  $$
  \text{score}_j = \sum_{l=1}^{12} w_l \cdot \text{sim}_{j,l}.
  $$
  - Chọn **Top-K** theo $\text{score}_j$.

- **Quy trình hoạt động**
  1. **Tải trọng số** lớp và **DB Train** (vectors + metadata), chuyển DB lên **GPU** nếu có.
  2. Nhập đường dẫn **file Span AV Test** → kiểm tra đủ **12 lớp**.
  3. **Stack 12 lớp** của test thành tensor `[12, 2304]`.
  4. Tính **cosine theo lớp** với toàn bộ DB → nhân trọng số → tổng thành **điểm**.
  5. Lấy **Top-K**; với mỗi neighbor:
     - Dựa vào `original_file` (regex `sentence_(\d+)_`) → lấy **sentence_idx**.
     - Mở đúng JSON Train gốc:  
       `<train_root>/<verb>_train_set.json` → lấy **sentence_text** tại `sentence_idx`.
  6. In kết quả ra màn hình.

In [1]:
# Import các thư viện cần thiết
import torch
import os
import json
import glob
from tqdm.auto import tqdm
import torch.nn.functional as F
import re

# --- 1. CẤU HÌNH ---
K_NEIGHBORS = 5
NUM_LAYERS_TO_MATCH = 12
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
output_dir = os.path.join(drive_base_path, 'matching_results')

# Đường dẫn đến file trọng số đã học
weights_file_path = os.path.join(output_dir, 'learned_layer_weights.pt')

# Đường dẫn đến database đã lưu
db_cache_dir = os.path.join(drive_base_path, 'cached_databases')
db_vectors_file = os.path.join(db_cache_dir, 'combined_train_db_vectors.pt')
db_metadata_file = os.path.join(db_cache_dir, 'combined_train_db_metadata.json')

# Đường dẫn đến thư mục dữ liệu gốc của tập TRAIN (để tra cứu text neighbor)
train_data_dir_gramvar = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train')
train_data_dir_parave = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train')
train_data_dirs_map = {
    'span_adaptation_vectors_train_gramvar': train_data_dir_gramvar,
    'span_adaptation_vectors_train_parave': train_data_dir_parave,
    # Thêm các nguồn dataset khác nếu DB của bạn bao gồm chúng
}

# Regex để trích xuất chỉ số câu
sentence_idx_pattern = re.compile(r"sentence_(\d+)_")

# --- 2. CÁC HÀM XỬ LÝ ---

def find_k_nearest_neighbors_weighted(test_span_vectors, db_vectors, db_metadata, k, weights):
    """Tìm kNN có trọng số."""
    test_vectors_unsqueezed = test_span_vectors.unsqueeze(0)
    weights = weights.to(db_vectors.device)
    similarities_per_layer = F.cosine_similarity(test_vectors_unsqueezed, db_vectors, dim=2)
    weighted_scores = torch.sum(similarities_per_layer * weights, dim=1)
    top_k_scores, top_k_indices = torch.topk(weighted_scores, k)

    neighbors = []
    for score, index in zip(top_k_scores, top_k_indices.cpu()):
        neighbor_info = db_metadata[index].copy()
        neighbor_info['score'] = score.item()
        neighbors.append(neighbor_info)
    return neighbors

def get_sentence_text_from_original(verb_name, sentence_idx, source_dataset_name):
    """Lấy text câu gốc của neighbor."""
    original_data_dir = train_data_dirs_map.get(source_dataset_name)
    if not original_data_dir: return "N/A (Nguồn không xác định)"

    original_data_path = os.path.join(original_data_dir, f"{verb_name}_train_set.json")
    try:
        # Tối ưu: Cache lại file json đã đọc nếu cần
        # (Trong ví dụ này, đọc lại mỗi lần cho đơn giản)
        with open(original_data_path, 'r', encoding='utf-8') as f:
            original_data = json.load(f)
            if sentence_idx < len(original_data):
                return original_data[sentence_idx].get('text', 'N/A')
            else: return "N/A (Index lỗi)"
    except Exception: return "N/A (Lỗi đọc file)"

# --- 3. TẢI DỮ LIỆU CỐ ĐỊNH ---
try:
    print("--- Bước 1: Tải trọng số quan trọng của các lớp ---")
    learned_weights = torch.load(weights_file_path, map_location=device)
    print(" -> Tải trọng số thành công!")
except Exception as e:
    print(f"LỖI: Không thể tải file trọng số '{weights_file_path}': {e}")
    exit()

print("\n--- Bước 2: Tải cơ sở dữ liệu Train ---")
try:
    train_db_vectors = torch.load(db_vectors_file)
    with open(db_metadata_file, 'r', encoding='utf-8') as f:
        train_db_metadata = json.load(f)
    print(f" -> Tải thành công DB với {len(train_db_metadata)} span.")
    print(f"Chuyển DB sang '{device}'...")
    train_db_vectors = train_db_vectors.to(device)
    print(" -> Chuyển thành công!")
except Exception as e:
    print(f"LỖI khi tải database: {e}.")
    print("Vui lòng chạy lại kịch bản 'build_database.py' trước.")
    exit()

# --- 4. VÒNG LẶP TƯƠNG TÁC ---
print("\n--- Bắt đầu chế độ tìm kiếm tương tác ---")
print("Nhập đường dẫn đến file Span AV (.pt) của mẫu test bạn muốn tìm.")
print("Nhấn Enter hoặc gõ 'quit' để thoát.")

while True:
    try:
        test_file_path = input("\nNhập đường dẫn file test Span AV: ").strip()
        if not test_file_path or test_file_path.lower() == 'quit':
            break

        if not os.path.exists(test_file_path):
            print(f"LỖI: File không tồn tại: {test_file_path}")
            continue

        # Trích xuất thông tin từ tên file
        filename_parts = os.path.basename(test_file_path).replace('.pt', '').split('_')
        arg_label = f"{filename_parts[-2]}-{filename_parts[-1]}"
        verb_name = os.path.basename(os.path.dirname(test_file_path))
        match = sentence_idx_pattern.search(os.path.basename(test_file_path))
        if not match:
            print("LỖI: Tên file không đúng định dạng (không tìm thấy 'sentence_..._').")
            continue
        sentence_idx = int(match.group(1))

        # Tải vector test
        layers_data = torch.load(test_file_path, map_location='cpu')
        if not all(f'layer_{j+1}' in layers_data for j in range(NUM_LAYERS_TO_MATCH)):
            print("LỖI: File vector test không chứa đủ 12 lớp.")
            continue

        layer_vectors = [layers_data[f'layer_{j+1}'] for j in range(NUM_LAYERS_TO_MATCH)]
        test_span_vectors = torch.stack(layer_vectors).to(device)

        # Tìm kiếm láng giềng
        neighbors = find_k_nearest_neighbors_weighted(
            test_span_vectors, train_db_vectors, train_db_metadata, K_NEIGHBORS, learned_weights
        )

        # In kết quả
        print(f"\n--- Kết quả tìm kiếm cho: {os.path.basename(test_file_path)} ---")
        print(f"Động từ: {verb_name}, Argument: {arg_label}")

        # (Tùy chọn) In câu gốc của mẫu test nếu muốn
        # test_sentence_text = get_sentence_text_from_test(...) # Cần thêm hàm tương tự cho test
        # print(f"Câu test gốc: {test_sentence_text}")

        print(f"\n{K_NEIGHBORS} láng giềng gần nhất trong tập Train:")
        for rank, neighbor in enumerate(neighbors, 1):
            neighbor_verb = neighbor['verb_name']
            neighbor_file = neighbor['original_file']
            source_dataset = neighbor.get('source_dataset', 'N/A') # Lấy nguồn nếu có

            neighbor_match = sentence_idx_pattern.search(neighbor_file)
            neighbor_text = "N/A"
            if neighbor_match:
                neighbor_idx = int(neighbor_match.group(1))
                neighbor_text = get_sentence_text_from_original(neighbor_verb, neighbor_idx, source_dataset)

            print(f"\n{rank}. Score: {neighbor['score']:.4f}")
            print(f"   Nguồn: {source_dataset} / {neighbor_verb} / {neighbor['arg_label']}")
            print(f"   Câu gốc: {neighbor_text}")
            print(f"   File vector gốc: {neighbor_file}")

    except KeyboardInterrupt:
        print("\nĐã dừng bởi người dùng.")
        break
    except Exception as e:
        print(f"\nĐã xảy ra lỗi: {e}")

print("\n--- Kết thúc chương trình tìm kiếm ---")


Sử dụng thiết bị: cpu
--- Bước 1: Tải trọng số quan trọng của các lớp ---
 -> Tải trọng số thành công!

--- Bước 2: Tải cơ sở dữ liệu Train ---
 -> Tải thành công DB với 15521 span.
Chuyển DB sang 'cpu'...
 -> Chuyển thành công!

--- Bắt đầu chế độ tìm kiếm tương tác ---
Nhập đường dẫn đến file Span AV (.pt) của mẫu test bạn muốn tìm.
Nhấn Enter hoặc gõ 'quit' để thoát.

Nhập đường dẫn file test Span AV: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_train_gramvar/abolish/sentence_1_ARG_0.pt

--- Kết quả tìm kiếm cho: sentence_1_ARG_0.pt ---
Động từ: abolish, Argument: ARG-0

5 láng giềng gần nhất trong tập Train:

1. Score: 1.0000
   Nguồn: span_adaptation_vectors_train_gramvar / abolish / ARG-0
   Câu gốc: One mutation abolishes RNA expression from the altered allele.
   File vector gốc: sentence_1_ARG_0.pt

2. Score: 0.9747
   Nguồn: span_adaptation_vectors_train_gramvar / abolish / ARG-0
   Câu gốc: One mutation can abolish RNA expression from t

- **Thông số chính**
  - `K_NEIGHBORS = 5`
  - `NUM_LAYERS_TO_MATCH = 12`
  - Thiết bị: `cuda` nếu có, ngược lại `cpu`.


- **Lưu ý/khuyến nghị**
  - Tên file phải đúng mẫu: `sentence_<idx>_ARG_<k>.pt` để regex trích **index**.
  - Nếu DB lớn, lần đầu **load** sẽ tốn RAM; nên dùng **DB đã cache** như trên.
  - Bổ sung thêm vào `train_data_dirs_map` nếu metadata DB có nguồn khác.
  - Có thể thêm hàm phụ để in luôn **câu test** (tương tự phần tra cứu Train).